In [4]:
import os
import pandas as pd

# get all json files that store our interactions with chatgpt
# these are the final, optimized files
json_files_final_run = [
    os.path.join(dp, f) for dp, dn, fn in os.walk("../data/prompts_new_format") for f in fn
    if ("final" in f) and f.endswith(".json")]

# json_files_final_run += [
#     os.path.join(dp, f) for dp, dn, fn in os.walk("../data/prompts_not_used") for f in fn
#     if ("final" in f or "try_4" in f) and f.endswith(".json")]

total_files = len(json_files_final_run)

# read all the data into a pandas dataframe
li = []
for filename in json_files_final_run:
    df = pd.read_json(filename)
    li.append(df)
frame_all_runs = pd.concat(li, axis=0, ignore_index=True)

In [5]:
frame_all_runs

,prompt,code,tfsec,number_of_issues
0,Can you create terraform configuration code in...,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",2
1,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": []}",0
2,Can you create terraform configuration code in...,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": []}",0
3,Can you create terraform configuration code in...,terraform {\n required_providers {\n aws =...,"{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",2
4,terraform {\n required_providers {\n aws =...,no code from chatgpt,,
...,...,...,...,...
272,Can you create terraform configuration code in...,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",2
273,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",3
274,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",1
275,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",1


In [6]:
import json
issues_change_dict = []
security_rule_ids_before = []
security_rule_ids_after = []
security_rule_per_file_before = []
security_rule_per_file_after = []
for file in json_files_final_run:
    json_data = json.load(open(file))
    iteration_0 = json_data[0]
    iteration_final = json_data[-1]
    iteration_final["number_of_issues"] = "X" if iteration_final["number_of_issues"] == "" else iteration_final["number_of_issues"]
    issues_change_dict.append(
        {"before": iteration_0["number_of_issues"],
         "after": iteration_final["number_of_issues"]}
    )
    tf_sec_result_json = json.loads(iteration_0["tfsec"])
    if tf_sec_result_json is not None:
        if tf_sec_result_json["results"]:
            for result in tf_sec_result_json["results"]:
                if "long_id" in result:
                    print(result["long_id"])
                    security_rule_per_file_before.append(
                        {"file": file, "rule_id": result["long_id"]}
                    )
                    security_rule_ids_before += [result["long_id"]]
    try:
        tf_sec_result_json = json.loads(iteration_final["tfsec"])
        if tf_sec_result_json is not None:
            if tf_sec_result_json["results"]:
                for result in tf_sec_result_json["results"]:
                    if "long_id" in result:
                        print(result["long_id"])
                        security_rule_per_file_after.append(
                        {"file": file, "rule_id": result["long_id"]}
                        )
                        security_rule_ids_after+= [result["long_id"]]
    except Exception as e:
        print(e)
        continue

aws-ec2-enable-at-rest-encryption
aws-ec2-enforce-http-token-imds
aws-ec2-enable-at-rest-encryption
aws-ec2-enforce-http-token-imds
Expecting value: line 1 column 1 (char 0)
aws-ec2-enable-at-rest-encryption
aws-ec2-enforce-http-token-imds
Expecting value: line 1 column 1 (char 0)
aws-ec2-enable-at-rest-encryption
aws-ec2-enforce-http-token-imds
Expecting value: line 1 column 1 (char 0)
aws-ec2-enable-at-rest-encryption
aws-ec2-enforce-http-token-imds
aws-s3-block-public-acls
aws-s3-block-public-acls
aws-s3-block-public-policy
aws-s3-block-public-policy
aws-s3-enable-bucket-encryption
aws-s3-enable-bucket-encryption
aws-s3-enable-bucket-logging
aws-s3-enable-bucket-logging
aws-s3-enable-versioning
aws-s3-enable-versioning
aws-s3-encryption-customer-key
aws-s3-encryption-customer-key
aws-s3-ignore-public-acls
aws-s3-ignore-public-acls
aws-s3-no-public-buckets
aws-s3-no-public-buckets
aws-s3-specify-public-access-block
aws-s3-specify-public-access-block
Expecting value: line 1 column 1 (

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
issues_change_dict

[{'before': '2', 'after': '0'},
 {'before': '0', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '20', 'after': 'X'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': '0'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '2', 'after': 'X'},
 {'before': '9', 'after': '0'},
 {'before': '7', 'after': 'X'},
 {'before': 'X', 'after': 'X'}]

In [8]:
import pandas as pd
df = pd.DataFrame(issues_change_dict)

In [9]:
df

,before,after
0,2,0
1,0,0
2,2,X
3,2,X
4,2,X
5,20,X
6,2,0
7,2,0
8,2,X
9,2,X


In [10]:
df.to_csv(path_or_buf="../data/number_of_issues_before_after.csv", header=True)

In [11]:
df_issues_before = pd.DataFrame(security_rule_ids_before, columns=["rule_id"])
df_issues_before["count"] = 1
df_issues_after = pd.DataFrame(security_rule_ids_after, columns=["rule_id"])
df_issues_after["count"] = 1

In [12]:
df_issues_after.groupby(by="rule_id").sum().sort_values("count").reset_index()

,rule_id,count


In [13]:
df_issues_before.groupby(by="rule_id").sum().sort_values("count").reset_index()

,rule_id,count
0,aws-s3-enable-bucket-encryption,3
1,aws-s3-enable-versioning,3
2,aws-s3-block-public-acls,4
3,aws-s3-block-public-policy,4
4,aws-s3-enable-bucket-logging,4
5,aws-s3-encryption-customer-key,4
6,aws-s3-ignore-public-acls,4
7,aws-s3-no-public-buckets,4
8,aws-s3-specify-public-access-block,4
9,aws-ec2-enable-at-rest-encryption,22


In [14]:
df = pd.DataFrame(security_rule_per_file_before, columns=["file", "rule_id"])
df.to_csv(path_or_buf="../data/issues_per_file_non_optimized.csv")

In [15]:
df = pd.DataFrame(security_rule_per_file_after, columns=["file", "rule_id"])
df.to_csv(path_or_buf="../data/issues_per_file_optimized.csv")